In [1]:
import wandb
wandb.login(key="c26df6b59bfb128917e73bbb00a79ca7e9324a11")


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: kokyloka2003 (kokyloka2003-msa) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [2]:
import torch
import numpy as np
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix, classification_report
from transformers import AutoTokenizer, AutoModel, Trainer, TrainingArguments
from datasets import Dataset
import logging
import shutil
import os

# Enable logging
logging.basicConfig(level=logging.INFO)

# Load full dataset
train_path = "/kaggle/input/maqa-unbalanced-with-severity/MAQA_Severity_Train.xlsx"
test_path = "/kaggle/input/maqa-unbalanced-with-severity/MAQA_Severity_Test.xlsx"
train_df = pd.read_excel(train_path)
test_df = pd.read_excel(test_path)
all_data = pd.concat([train_df, test_df], ignore_index=True)
all_data = all_data[['q_body', 'category', 'severity']]

# Valid categories and severity levels
valid_categories = [
    "امراض نسائية",
    "امراض العضلات والعظام و المفاصل",
    "امراض الجهاز الهضمي",
    "الامراض الجنسية",
    "طب الاسنان",
    "امراض القلب و الشرايين",
    "امراض العيون",
    "انف اذن وحنجرة",
    "جراحة تجميل",
    "امراض الدم"
]
valid_severity = ["حرج", "غير حرج"]

# Filter dataset
all_data = all_data[all_data["category"].isin(valid_categories) & all_data["severity"].isin(valid_severity)]
all_data = all_data.sample(frac=1, random_state=42).reset_index(drop=True)

# Split into train and test
train_df, test_df = train_test_split(all_data, test_size=0.2, random_state=42, stratify=all_data["severity"])

# Map severity to labels
severity_mapping = {sev: i for i, sev in enumerate(valid_severity)}
train_df['label'] = train_df['severity'].map(severity_mapping)
test_df['label'] = test_df['severity'].map(severity_mapping)

# Load model & tokenizer
model_name = "monologg/biobert_v1.1_pubmed"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenization
def tokenize_function(examples):
    return tokenizer(examples["q_body"], padding="max_length", truncation=True, max_length=128)

train_dataset = Dataset.from_pandas(train_df[['q_body', 'label']])
test_dataset = Dataset.from_pandas(test_df[['q_body', 'label']])
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Define custom model
base_model = AutoModel.from_pretrained(model_name)

class CustomModel(torch.nn.Module):
    def __init__(self, base_model, num_labels):
        super(CustomModel, self).__init__()
        self.base_model = base_model
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(base_model.config.hidden_size, num_labels)
        self.loss_fn = torch.nn.CrossEntropyLoss()

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.base_model(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0, :]
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        loss = self.loss_fn(logits, labels) if labels is not None else None
        return {"loss": loss, "logits": logits}

model = CustomModel(base_model, num_labels=len(valid_severity))

# Training args
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    logging_dir="./logs",
    logging_steps=10,
    per_device_train_batch_size=96,
    per_device_eval_batch_size=96,
    num_train_epochs=10,
    weight_decay=0.01,
    learning_rate=3e-5,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)

# Metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="weighted")
    acc = accuracy_score(labels, predictions)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Train
print("\n🔹 Starting Training...")
trainer.train()

# Evaluate
test_metrics = trainer.evaluate(test_dataset)
print("\n🔹 Test Metrics:", test_metrics)

# Predict
predictions = trainer.predict(test_dataset)
preds = np.argmax(predictions.predictions, axis=1)
labels = predictions.label_ids

print("\n🔹 Confusion Matrix:")
print(confusion_matrix(labels, preds))
print("\n🔹 Classification Report:")
print(classification_report(labels, preds))

# Save model
def save_complete_model(model, tokenizer, severity_mapping, save_path):
    model.base_model.save_pretrained(save_path)
    tokenizer.save_pretrained(save_path)
    torch.save({
        'classifier_state': model.classifier.state_dict(),
        'num_labels': model.classifier.out_features
    }, f"{save_path}/classifier_state.pt")
    with open(f"{save_path}/severity_mapping.pkl", "wb") as f:
        pickle.dump(severity_mapping, f)

save_complete_model(trainer.model, tokenizer, severity_mapping, "BioBERT-Severity")

# Load model
def load_complete_model(model_path):
    base_model = AutoModel.from_pretrained(model_path)
    classifier_state = torch.load(f"{model_path}/classifier_state.pt", map_location=torch.device('cpu'))
    model = CustomModel(base_model, classifier_state['num_labels'])
    model.classifier.load_state_dict(classifier_state['classifier_state'])
    model.eval()
    return model

tokenizer = AutoTokenizer.from_pretrained("BioBERT-Severity")
model2 = load_complete_model("BioBERT-Severity")
print("✅ Model Loaded Successfully!")

# Prediction function
def predict_severity(text, model, tokenizer, severity_mapping):
    inputs = tokenizer(text, padding="max_length", truncation=True, max_length=128, return_tensors="pt")
    with torch.no_grad():
        outputs = model(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"])
    logits = outputs["logits"]
    predicted_label = torch.argmax(logits, dim=-1).item()
    severity_mapping_reverse = {v: k for k, v in severity_mapping.items()}
    return severity_mapping_reverse[predicted_label]

# Test predictions
test_samples = ["الاعراض شديده", "تحتاج الى راحه فقط", "حاله متوسطه", "نزيف حاد"]
for text in test_samples:
    predicted_severity = predict_severity(text, model2, tokenizer, severity_mapping)
    print(f"\n🔹 Input: {text}")
    print(f"Predicted Severity: {predicted_severity}")

# Zip model
shutil.make_archive("BioBERT-Severity", 'zip', "BioBERT-Severity")
print("\n✅ Model Saved & Zipped for Download!")

# Download link
from IPython.display import FileLink
FileLink(r'BioBERT-Severity.zip')


2025-06-16 19:23:54.991729: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750101835.184806      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750101835.239666      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/433 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

...

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked t


🔹 Test Metrics: {'eval_loss': 0.4123944044113159, 'eval_accuracy': 0.8242346554064244, 'eval_precision': 0.8321127313546847, 'eval_recall': 0.8242346554064244, 'eval_f1': 0.8274218586640452, 'eval_runtime': 177.3038, 'eval_samples_per_second': 224.395, 'eval_steps_per_second': 1.173, 'epoch': 10.0}


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(



🔹 Confusion Matrix:
[[26006  4069]
 [ 2924  6787]]

🔹 Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.86      0.88     30075
           1       0.63      0.70      0.66      9711

    accuracy                           0.82     39786
   macro avg       0.76      0.78      0.77     39786
weighted avg       0.83      0.82      0.83     39786

✅ Model Loaded Successfully!

🔹 Input: الاعراض شديده
Predicted Severity: حرج

🔹 Input: تحتاج الى راحه فقط
Predicted Severity: حرج

🔹 Input: حاله متوسطه
Predicted Severity: حرج

🔹 Input: نزيف حاد
Predicted Severity: حرج

✅ Model Saved & Zipped for Download!


/kaggle/working/BioBERT-Severity.zip

In [3]:
from pprint import pprint
report = classification_report(labels, preds, output_dict=True)
print("\n🔹 Classification Report (high precision):")
print(f"Accuracy: {accuracy_score(labels, preds):.4f}")
print(f"Macro Avg F1-score: {report['macro avg']['f1-score']:.4f}")
print(f"Weighted Avg F1-score: {report['weighted avg']['f1-score']:.4f}")
print("\n🔹 Full Report Dictionary:")
pprint(report)



🔹 Classification Report (high precision):
Accuracy: 0.8242
Macro Avg F1-score: 0.7707
Weighted Avg F1-score: 0.8274

🔹 Full Report Dictionary:
{'0': {'f1-score': 0.8814846199474621,
       'precision': 0.8989284479778776,
       'recall': 0.8647049044056525,
       'support': 30075},
 '1': {'f1-score': 0.6599893032527836,
       'precision': 0.6251842299189388,
       'recall': 0.6988981567294821,
       'support': 9711},
 'accuracy': 0.8242346554064244,
 'macro avg': {'f1-score': 0.7707369616001228,
               'precision': 0.7620563389484083,
               'recall': 0.7818015305675673,
               'support': 39786},
 'weighted avg': {'f1-score': 0.8274218586640452,
                  'precision': 0.8321127313546847,
                  'recall': 0.8242346554064244,
                  'support': 39786}}
